<a href="https://colab.research.google.com/github/nicolenair/capstone/blob/master/Capstone_Classification_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
# !pip install importlib
# !pip install imp

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Colab Notebooks")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#!pip install import_ipynb

from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import os
import re
import os
import importlib

In [0]:
# importlib.reload(anno_auto)

In [0]:
# import os
# os.listdir("/content/gdrive/My Drive/Nicole-Internship-Complete-Folder/chatbot - all/chatbot-flask/GoogleNews-vectors-negative300.bin.gz")

In [0]:
books = ["gables", "dreams", "island", "avonlea", "poplars" ,"ingleside", "rainbow", "rilla"]
# urls = ["https://www.gutenberg.org/files/45/45-0.txt", "https://www.gutenberg.org/files/47/47-0.txt", "https://www.gutenberg.org/files/51/51-0.txt", "http://www.gutenberg.org/cache/epub/544/pg544.txt", "http://www.gutenberg.org/cache/epub/3796/pg3796.txt", "http://gutenberg.net.au/ebooks01/0100281.txt", "http://gutenberg.net.au/ebooks01/0100251.txt", "http://www.gutenberg.org/files/5343/5343-0.txt"]

# for i in range(len(books)):
#     page = requests.get(urls[i])
#     file = open("{}.txt".format(books[i]) , "w+")
#     file.write(page.text)
#     file.close()



file = open("gables.txt", "r")
gables = file.read()
file = open("avonlea.txt", "r")
avonlea = file.read()
file = open("island.txt", "r")
island = file.read()
file = open("dreams.txt", "r")
dreams = file.read()
file = open("rilla.txt", "r")
rilla = file.read()
file = open("ingleside.txt", "r")
ingleside = file.read()
file = open("poplars.txt", "r")
poplars = file.read()
file = open("rainbow.txt", "r")
rainbow = file.read()

In [0]:
# import gensim
# must download & load for word2vec_average_doc function to work
# # Load Google's pre-trained Word2Vec model.
# model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)  

In [0]:
# from gensim.parsing.preprocessing import preprocess_documents
# from gensim.models import KeyedVectors
# from gensim.models import Word2Vec

# #returns vector representation of documents 
# def word2vec_average_doc(documents):
#     model_new = Word2Vec(size=300, min_count=1)
#     model_new.build_vocab(documents)
#     total_examples = model_new.corpus_count
#     model_new.build_vocab([list(model_new.wv.vocab.keys())], update=True) #fishy here, changed model to model_new within list bracket
#     model_new.intersect_word2vec_format("/content/gdrive/My Drive/Nicole-Internship-Complete-Folder/chatbot - all/chatbot-flask/GoogleNews-vectors-negative300.bin.gz", binary = True)
#     model_new.train(documents, total_examples=total_examples, epochs=model_new.iter)
#     mean_vectors = []
#     processed = preprocess_documents(documents)
#     for i in processed:
#         summation = []
#         for e in i:
#             try:
#                 summation.append(model_new[e])
#             except KeyError:
#                 #print("debug")
#                 next
#         if len(summation)!=0:
#             mean_vectors.append(np.mean(summation, axis = 0))
#         else:
#             mean_vectors.append(np.mean(mean_vectors, axis = 0))
        
#         #print(summation)
#     return np.array(mean_vectors)

In [0]:
file1 = open("gables_sentence_list.txt", "r")
file2 = open("dreams_sentence_list.txt", "r")
file3 = open("island_sentence_list.txt", "r")
file4 = open("avonlea_sentence_list.txt", "r")
file5 = open("poplars_sentence_list.txt", "r")
file6 = open("ingleside_sentence_list.txt", "r")
file7 = open("rainbow_sentence_list.txt", "r")
file8 = open("rilla_sentence_list.txt", "r")
sentence_gables = []
sentence_dreams = []
sentence_island = []
sentence_avonlea = []
sentence_poplars = []
sentence_ingleside = []
sentence_rainbow = []
sentence_rilla = []
target_gables = []
target_dreams = []
target_island = []
target_avonlea = [] ###
target_poplars = []
target_ingleside = []
target_rainbow = []
target_rilla = [] ###

file1_lines =file1.readlines()
file2_lines = file2.readlines()
file3_lines = file3.readlines()
file4_lines = file4.readlines()
file5_lines = file5.readlines()
file6_lines = file6.readlines()
file7_lines = file7.readlines()
file8_lines = file8.readlines()

files = [file1_lines, file2_lines, file3_lines ,file4_lines, file5_lines, file6_lines, file7_lines, file8_lines]
sentences = [sentence_gables, sentence_dreams, sentence_island, sentence_avonlea, sentence_poplars, sentence_ingleside, sentence_rainbow, sentence_rilla]
targets = [target_gables, target_dreams, target_island, target_avonlea, target_poplars, target_ingleside, target_rainbow, target_rilla]

for s in range(len(sentences)):
  for i in range(len(files[s])):
    if i%2==0:
      sentences[s].append([files[s][i]])
    else:
      targets[s].append([int(files[s][i])])
  file.close()

    
# for i in range(len(file2_lines)):
#   if i%2==0:
#     sentence_dreams.append([file2_lines[i]])
#   else:
#     target_dreams.append([int(file2_lines[i])])
# file.close()

In [0]:
full_corpus = []

for i in sentences:
  full_corpus.extend(i)

In [0]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(full_corpus)]
model_d2v = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix
from random import sample
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix


#function for training classifier
def train_classifier(full_para_list, target, vectorizer=model_d2v, prnt = True, d2v=False):
    if d2v==False:
  
      vectorizer = CountVectorizer()
      n=len(target)

      x_lst = []
      for i in full_para_list[:n]:
          x_lst.extend(i) 
      vectorized = vectorizer.fit_transform(x_lst)
      x = vectorized.todense()
    else:
      n = len(target)
      x_lst = []
      for i in full_para_list[:n]:
        x_lst.extend(i)
      vectorized = np.array([vectorizer.infer_vector(i.split()) for i in x_lst])
      x = vectorized
#     x = word2vec_average_doc(x_lst) 
#     x = model_d2v.infer_vector(x_lst)
#     print(x)


    y = []
    for i in target[:n]:
        y.extend(i) 
    y = [0 if i!=1 else 1 for i in y]
    ind_0 = [e for e, i in enumerate(y) if i==0]
    ind_1 = [e for e, i in enumerate(y) if i==1]
#     print(ind_0)
#     print(ind_1)
    
        
    if len(ind_0)>len(ind_1):
        
        new_indices = random.sample(ind_0, len(ind_1))
#         print(new_indices)
        x = np.vstack([x[new_indices], x[ind_1]])
        y = [0 for i in range(len(new_indices))] + [1 for i in range(len(ind_1))]
    else:
        new_indices = sample(ind_1, len(ind_0))
        x = np.vstack([x[new_indices], x[ind_0]])
        y = [0 for i in range(len(new_indices))] + [1 for i in range(len(ind_0))]

#     print(x)
    model2 = LogisticRegression(solver='lbfgs', multi_class='multinomial')
   # X_train, X_test, y_train, y_test = train_test_split(x, y)
    model2.fit(x, y)
#     if prnt==True:
#         print(model2.score(X_test, y_test))
#         y_pred = model2.predict(X_test)
#         print(precision_score(y_test, y_pred,  pos_label = 1))
#         print(precision_score(y_test, y_pred,  pos_label = 0))
#         print(recall_score(y_test, y_pred, pos_label = 1))
#         print(recall_score(y_test, y_pred, pos_label = 0 ))
#         print(confusion_matrix(y_test, y_pred))
#         tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
#         print(tn, fp, fn, tp)
    return model2, vectorizer

In [0]:
import random

# for checking performance of trained classifiers on books
def test(full_para_list, model2, vectorizer, target, d2v=False):
    n = len(target)
#     print(n)
    x_lst = []
    for i in full_para_list:
        x_lst.extend(i) 
    if d2v!=True:
      vectorized = vectorizer.transform(x_lst)
      x = vectorized.todense()
    else:
      vectorized = [vectorizer.infer_vector(i.split()) for i in x_lst]
      x = np.array(vectorized)

    
#     x = word2vec_average_doc(x_lst) 
#     x = vectorizer.infer_vector(x_lst)
    y = []
    for i in target:
        y.extend(i) 

    y = [0 if i!=1 else 1 for i in y]
    ind_0 = [e for e, i in enumerate(y) if i==0]
    ind_1 = [e for e, i in enumerate(y) if i==1]

#     print(ind_0)
#     print(ind_1)
    if len(ind_0)>len(ind_1):
        new_indices = random.sample(ind_0, len(ind_1))
        #print(x.shape)
#         print(new_indices)
        x = np.vstack([x[new_indices], x[ind_1]])
        
        y = [0 for i in range(len(new_indices))] + [1 for i in range(len(ind_1))]
    else:
        new_indices = sample(ind_1, len(ind_0))
        x = np.vstack([x[new_indices], x[ind_0]])
        y = [0 for i in range(len(new_indices))] + [1 for i in range(len(ind_0))]

    print(model2.score(x, y))
    y_pred = model2.predict(x)
    print("precision, label = 1")
    print(precision_score(y, y_pred,  pos_label = 1))
    print("precision, label = 0")
    print(precision_score(y, y_pred,  pos_label = 0))
    print("recall, label = 1")
    print(recall_score(y, y_pred, pos_label = 1))
    print("recall, label = 0")
    print(recall_score(y, y_pred, pos_label = 0 ))

    print(confusion_matrix(y, y_pred))
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    print(tn, fp, fn, tp)
    print()

#     words = np.array(vectorizer.get_feature_names())
#     words[np.argsort(abs(model2.coef_[0]))[-10:]], model2.coef_[0][abs(np.argsort(model2.coef_[0]))[-10:]]
    
# print("trained on AGG, tested on AGG (control)")
# model2, vectorizer = train_classifier(full_para_list_gables, anno.return_target("gables"), prnt = False)
# test(full_para_list_gables, model2, vectorizer, anno.return_target("gables"))

# print("trained on AGG, tested on AHD")
# model2, vectorizer = train_classifier(full_para_list_gables, anno.return_target("gables"), prnt = False)
# test(full_para_list_dreams, model2, vectorizer, anno.return_target("dreams"))

# print("trained on AHD, tested on AHD (control)")
# model2, vectorizer = train_classifier(full_para_list_dreams,anno.return_target("dreams"), prnt = False)
# test(full_para_list_dreams, model2, vectorizer, anno.return_target("dreams"))

# print("trained on AHD, tested on AGG")
# model2, vectorizer = train_classifier(full_para_list_dreams, anno.return_target("dreams"), prnt = False)
# test(full_para_list_gables, model2, vectorizer, anno.return_target("dreams"))

In [0]:
# sentence_gables_Marilla, target_gables_Marilla = anno_auto.return_nsubj("gables", "Marilla")


In [14]:
#as compared to other nsubj sentences


for i in range(len(sentences)):
  for e in range(len(sentences)):
    X_train, X_test, y_train, y_test = train_test_split(sentences[i], targets[i])
    if i!=e:
      X_train_buff, X_test, y_train_buff, y_test = train_test_split(sentences[e], targets[e])

    print("trained on {}, tested on {}".format(books[i], books[e]))
    model2, vectorizer = train_classifier(X_train, y_train, prnt = False)
    test(X_test, model2, vectorizer, y_test)


trained on gables, tested on gables
0.7663043478260869
precision, label = 1
0.7474747474747475
precision, label = 0
0.788235294117647
recall, label = 1
0.8043478260869565
recall, label = 0
0.7282608695652174
[[67 25]
 [18 74]]
67 25 18 74

trained on gables, tested on dreams
0.7755102040816326
precision, label = 1
0.7872340425531915
precision, label = 0
0.7647058823529411
recall, label = 1
0.7551020408163265
recall, label = 0
0.7959183673469388
[[39 10]
 [12 37]]
39 10 12 37

trained on gables, tested on island
0.7131147540983607
precision, label = 1
0.7166666666666667
precision, label = 0
0.7096774193548387
recall, label = 1
0.7049180327868853
recall, label = 0
0.7213114754098361
[[44 17]
 [18 43]]
44 17 18 43

trained on gables, tested on avonlea
0.7628205128205128
precision, label = 1
0.759493670886076
precision, label = 0
0.7662337662337663
recall, label = 1
0.7692307692307693
recall, label = 0
0.7564102564102564
[[59 19]
 [18 60]]
59 19 18 60

trained on gables, tested on poplars


KeyboardInterrupt: ignored

In [17]:
len(gables)

595661

In [0]:
# #as compared to other nsubj sentences, using Marilla as comparison
# X_train, X_test, y_train, y_test = train_test_split(sentence_gables, target_gables)
# X_train_dreams, X_test_dreams, y_train_dreams, y_test_dreams = train_test_split(sentence_dreams, target_dreams)

# print("trained on AGG, tested on AGG")
# model2, vectorizer = train_classifier(X_train, y_train, prnt = False)
# test(sentence_gables, model2, vectorizer, target_gables)

# print("trained on AGG, tested on AGG (control)(Marilla)")
# model2, vectorizer = train_classifier(X_train, y_train, prnt = False)
# test(sentence_gables_Marilla, model2, vectorizer, target_gables_Marilla)

# print("trained on AHD, tested on AGG(Marilla) (control)")
# model2, vectorizer = train_classifier(X_train_dreams, y_train_dreams, prnt = False)
# test(sentence_gables_Marilla, model2, vectorizer, target_gables_Marilla)

# print("trained on AHD, tested on AHD")
# model2, vectorizer = train_classifier(X_train_dreams, y_train_dreams, prnt = False)
# test(X_test_dreams, model2, vectorizer, y_test_dreams)

# # print("trained on AGG, tested on AHD")
# # model2, vectorizer = train_classifier(X_train, y_train, prnt = False)
# # test(X_test_dreams, model2, vectorizer, y_test_dreams)

# # print("trained on AHD, tested on AHD (control)")
# # model2, vectorizer = train_classifier(X_train_dreams,y_train_dreams, prnt = False)
# # test(X_test_dreams, model2, vectorizer, y_test_dreams)

# # print("trained on AHD, tested on AGG")
# # model2, vectorizer = train_classifier(X_train_dreams,y_train_dreams, prnt = False)
# # test(X_test, model2, vectorizer, y_test)

In [0]:
import matplotlib.pyplot as plt
import networkx as nx

#function for visualizing character networks in a book
def gr(full_para_list):
    a = np.zeros((20, 20))

    x_lst = []
    for i in full_para_list:
        x_lst.extend(i) 
    # vectorized = vectorizer.transform(x_lst)

    for i in x_lst:
        chars = name_mapping.keys()
        dis_chars = []
        for ch in chars:
            if re.search(ch, i) and ch not in dis_chars:
                dis_chars.append(ch)
        for ch in dis_chars:
            for neighbor in dis_chars:
                a[name_mapping[ch]][name_mapping[neighbor]] += 1

    g1 = (nx.from_numpy_matrix((a- min(a.flatten()))/(max(a.flatten()) - min(a.flatten()))))
    #g1 = (nx.from_numpy_matrix((a)
    pos = nx.spring_layout(g1, k =1)
    weights = [g1[u][v]['weight'] for u,v in g1.edges]
    nx.draw(g1, pos = pos, width = weights)
    nx.draw_networkx_labels(g1, labels = dict(map(reversed, name_mapping.items())), pos = pos)
    plt.show()
    return a
    


In [0]:
# a_gables = gr(full_para_list_gables)
# a_dreams = gr(full_para_list_dreams)

# #printing co-occurrence values for Anne with Diana & Gilbert in AGG
# print(a_gables[1][6], a_gables[1][4])
# #printing co-occurrence values for Anne with Cornelia & Gilbert in AHD
# print(a_dreams[1][15], a_dreams[1][4])

In [0]:
# train_classifier(full_para_list_gables, target_gables, prnt = False)

In [0]:
import anno